## Neural Networks

To obtain higher classification accuracy, we implemented neural networks, which we did not learn in the class. Multi-layer perceptrons neural network is a supervised method, and is very powerful in classifying Alzheimer's disease as shown below.

In [1]:
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [2]:
# load data
df_train = pd.read_csv("data/ADNIMERGE_train.csv")
df_test = pd.read_csv("data/ADNIMERGE_test.csv")
X_train = df_train.drop(['RID', 'DX_bl'], axis=1).copy()
y_train = df_train['DX_bl'].copy()
X_test = df_test.drop(['RID', 'DX_bl'], axis=1).copy()
y_test = df_test['DX_bl'].copy()

In [3]:
# function to help compare the accuracy of models
def score(model, X_train, y_train, X_test, y_test):
    train_acc = model.score(X_train,y_train)
    test_acc = model.score(X_test,y_test)
    test_class0 = model.score(X_test[y_test==0], y_test[y_test==0])
    test_class1 = model.score(X_test[y_test==1], y_test[y_test==1])
    test_class2 = model.score(X_test[y_test==2], y_test[y_test==2])
    return pd.Series([train_acc, test_acc, test_class0, test_class1, test_class2],
                    index = ['Train accuracy', 'Test accuracy', 
                             "Test accuracy CN", "Test accuracy CI", "Test accuracy AD"])

In [4]:
cols_standardize = [
    c for c in X_train.columns 
    if (not c.startswith('PT')) \
        or (c=='PTEDUCAT') or (c=='PTAGE')]

X_train_std = X_train.copy()
X_test_std = X_test.copy()
for c in cols_standardize:
    col_mean = np.mean(X_train[c])
    col_sd = np.std(X_train[c])
    if col_sd > (1e-10)*col_mean:
        X_train_std[c] = (X_train[c]-col_mean)/col_sd
        X_test_std[c] = (X_test[c]-col_mean)/col_sd

In [5]:
print(X_train_std.shape)
X_train_std.head()

(621, 75)


,PTAGE,PTGENDER,PTEDUCAT,PTRACCAT_Asian,PTRACCAT_Black,PTRACCAT_Hawaiian/Other_PI,PTRACCAT_More_than_one,PTRACCAT_Unknown,PTRACCAT_White,PTETHCAT_Not_Hisp/Latino,PTMARRY_Married,PTMARRY_Never_married,PTMARRY_Widowed,APOE4,CSF_ABETA,CSF_TAU,CSF_PTAU,FDG,FDG_slope,AV45,AV45_slope,ADAS13,ADAS13_slope,MMSE,MMSE_slope,RAVLT_immediate,RAVLT_immediate_slope,RAVLT_learning,RAVLT_learning_slope,RAVLT_forgetting,RAVLT_forgetting_slope,RAVLT_perc_forgetting,RAVLT_perc_forgetting_slope,MOCA,MOCA_slope,EcogPtMem,EcogPtMem_slope,EcogPtLang,EcogPtLang_slope,EcogPtVisspat,EcogPtVisspat_slope,EcogPtPlan,EcogPtPlan_slope,EcogPtOrgan,EcogPtOrgan_slope,EcogPtDivatt,EcogPtDivatt_slope,EcogSPMem,EcogSPMem_slope,EcogSPLang,EcogSPLang_slope,EcogSPVisspat,EcogSPVisspat_slope,EcogSPPlan,EcogSPPlan_slope,EcogSPOrgan,EcogSPOrgan_slope,EcogSPDivatt,EcogSPDivatt_slope,FAQ,FAQ_slope,Ventricles,Ventricles_slope,Hippocampus,Hippocampus_slope,WholeBrain,WholeBrain_slope,Entorhinal,Entorhinal_slope,Fusiform,Fusiform_slope,MidTemp,MidTemp_slope,ICV,ICV_slope
0,2.208480,0,-2.852257,0,0,0,0,0,1,1,0,0,1,-0.823084,-1.421715,1.137347,-0.321586,-0.833130,0.811782,1.257634,0.023763,2.140034,0.236256,-2.742373,-0.471665,-0.010026,-0.001926,0.003844,-0.000699,0.028414,-0.018332,0.019484,-0.012471,-0.003150,-0.004052,-0.827740,1.136728,-0.981277,-0.002156,-0.715234,-0.030329,-0.713577,-0.002410,-0.843488,-0.007633,-1.076964,-0.016544,1.929076,-0.088208,-0.844339,0.954666,1.066767,1.048123,2.524016,0.060641,2.287022,0.021430,2.330806,-0.631242,2.934694,0.124173,-0.205919,0.381384,-1.351616,0.022285,-1.761500,-0.567555,-0.820814,-1.269796,-1.426968,0.156847,-2.102069,-0.192827,-1.574482,0.093937
1,0.759714,1,1.376909,0,0,0,0,0,1,1,1,0,0,0.703236,-0.567568,-0.086517,-0.128000,0.602793,2.864460,1.903103,1.199280,-0.210782,-0.364551,0.586715,0.595629,0.202187,-0.189405,0.884055,-0.907178,-0.118596,1.750365,-0.556812,1.770545,0.227228,-0.044392,-0.827740,-0.229483,-1.153377,-0.000458,-0.444998,-0.101471,-0.713577,-0.065235,-0.843488,-0.064451,-1.076964,-0.072474,0.573694,-0.738447,-0.106940,-1.151805,0.487637,-1.687565,1.262926,-2.477232,0.148179,-1.526413,0.971872,-1.384981,-0.484392,-0.424658,-0.053348,-0.251084,0.483644,-0.306894,-0.134464,-0.028641,-0.070387,0.188014,0.721399,-0.067438,0.019784,0.506511,-0.489132,-0.265646
2,-0.257208,0,0.607970,0,0,0,0,0,1,1,1,0,0,-0.823084,-0.775573,-1.038657,-0.920353,0.572072,0.019993,0.547072,0.023763,-0.798486,-0.710060,0.586715,0.463914,1.183334,1.011032,0.884055,-0.590705,0.280068,-1.497230,-0.495770,-1.722958,0.452935,0.701979,-0.530055,-0.049247,-0.464974,-0.085273,-0.715234,-0.033968,-0.713577,-0.065235,-0.018265,0.055382,-0.053520,0.069781,-1.052764,0.042214,-0.844339,-0.077852,-0.670610,-0.211993,-0.754818,0.111691,-0.824015,-0.212459,-0.930635,0.246899,-0.647205,-0.415515,-0.294977,-0.883418,-1.104796,0.020857,-1.300396,0.310720,0.456478,-0.560840,0.292776,0.016824,-0.650452,0.224140,-1.239633,-0.014198
3,-0.521887,0,-0.160970,0,0,0,0,0,1,1,0,0,0,-0.823084,-0.445863,0.222762,0.322200,0.441716,0.850276,-0.548541,-0.049974,-0.994388,-0.282924,0.586715,0.519651,0.730497,0.190865,-0.546632,0.454839,-0.915923,1.164759,-1.032937,0.965802,0.678642,0.204009,0.387783,0.167724,0.223444,0.375527,-0.174781,0.049689,-0.007973,0.191936,0.531879,0.404523,1.652218,0.250482,-0.103997,-0.010217,-0.844339,-0.186429,-0.670610,-0.271617,-0.754818,-0.343706,-0.824015,-0.076909,-0.115275,0.065754,-0.484392,-0.473929,-0.005181,-0.032439,0.013960,0.020857,-0.000094,-0.003749,0.006635,-0.003683,0.010325,0.015345,0.018697,0.004091,-0.005136,0.004314
4,1.121905,1,-0.160970,0,0,0,0,0,1,1,0,0,1,-0.823084,-1.521292,0.516578,0.056582,0.613315,1.113319,1.781157,0.338415,-0.602585,-0.189067,0.586715,0.476561,-0.099704,0.488832,-0.546632,0.189374,-0.118596,0.780831,-0.160039,0.662249,-0.449892,0.202553,-0.133156,0.119957,-0.809176,0.038450,-0.444998,-0.146690,-0.360775,-0.127391,0.806959,-0.571339,0.287627,-0.263162,0.709232,0.108071,-0.401908,0.043380,-0.27350

In [6]:
# find the best parameters
cv_fold = KFold(n_splits=3, shuffle=True, random_state=9001)
parameters = {'alpha': [1e-3, 1e-2, 1e-1, 0.3, 1, 3, 1e1, 1e2, 1e3],
              'hidden_layer_sizes': [(50), (100), (200), (500), 
                                     (50, 10), (50, 25), 
                                     (100, 10)]}
mlp = MLPClassifier(solver='lbfgs', activation='logistic', random_state=9001)
mlp_cv = GridSearchCV(mlp, parameters, cv=cv_fold)
mlp_cv.fit(X_train_std, y_train)
best_score = np.argmax(mlp_cv.cv_results_['mean_test_score'])
result = mlp_cv.cv_results_['params'][best_score]
a = result['alpha']
hidden_layer = result['hidden_layer_sizes']
mlp = MLPClassifier(solver='lbfgs', activation='logistic', random_state=9001,
                    alpha = a, hidden_layer_sizes=hidden_layer)
mlp = mlp.fit(X_train_std, y_train)

In [7]:
print("Optimal parameters")
print("L2 penalty parameter: ", a)
print("Hidden Layer Sizes: ", hidden_layer)
print('\n-----------------\n')
print("Training accuracy: ", mlp.score(X_train_std, y_train))
print("Test accuracy: ", mlp.score(X_test_std, y_test))
print('\n-----------------\n')
print('Test Confusion Matrix: ')
print(confusion_matrix(y_test, mlp.predict(X_test_std)))
nn_score = score(mlp, X_train_std, y_train, X_test_std, y_test)

Optimal parameters
L2 penalty parameter:  3
Hidden Layer Sizes:  200

-----------------

Training accuracy:  0.890499194847
Test accuracy:  0.827160493827

-----------------

Test Confusion Matrix: 
[[29 13  0]
 [ 7 83  3]
 [ 0  5 22]]


In [8]:
# random forest to compare with
rf_best = RandomForestClassifier(n_estimators=32, max_depth=12, random_state=9001)
rf_best.fit(X_train, y_train)
rf_score = score(rf_best, X_train, y_train, X_test, y_test)
print('\n-----------------\n')
print("Training accuracy: ", rf_best.score(X_train, y_train))
print("Test accuracy: ", rf_best.score(X_test, y_test))
print('\n-----------------\n')
print('Test Confusion Matrix: ')
print(confusion_matrix(y_test, rf_best.predict(X_test)))


-----------------

Training accuracy:  0.998389694042
Test accuracy:  0.802469135802

-----------------

Test Confusion Matrix: 
[[20 22  0]
 [ 5 85  3]
 [ 0  2 25]]


In [9]:
score_df = pd.DataFrame({"Neural Network": nn_score,
                         "Random Forest": rf_score})
score_df

,Neural Network,Random Forest
Train accuracy,0.890499,0.998390
Test accuracy,0.827160,0.802469
Test accuracy CN,0.690476,0.476190
Test accuracy CI,0.892473,0.913978
Test accuracy AD,0.814815,0.925926


The optimal hidden layer size is 1 hidden layer with 200 neurons. We need a l2-regularization term with value 3 to achieve the best accuracy.

The overall test accuracy of neural networks is better than that of the best random forest in the previous model comparison section. It also has a significantly higher accuracy on `CN`, while its accuracy on `CI` and `AD` is slightly lower. We would say that neural networks belong to the group of Type III classifiers discussed in the previous section. It is also a very promising model.